In [ ]:
# Imports we'll need...
from astropy.constants import G
import astropy.units as u
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# The unit system we'll use:
units = [u.Myr, u.kpc, u.Msun]

# Cython

Cython is a hybrid language that is a superset of Python---i.e. it supports all Python syntax---but adds new C-like syntax to support declaring types, directly interfacing with C (and C++) code, and wrapping C (and C++) code. Cython has to be converted to C and compiled before it can be used from Python, but that is taken care of by the Cython generator. The compilation can either be done manually (i.e. by executing a compiler, like `gcc` on the Cython output), or as a part of the build process of a package. 

Cython comes with an [IPython magic command](http://ipython.readthedocs.io/en/stable/interactive/magics.html) that allows you to write Cython code in a cell, which is then converted and compiled on the fly for you. In the tutorial today, we're going to go through each way of interacting with Cython, starting with this notebook.

## When to use Cython

Cython is a useful tool for optimizing Python code once you identify the bottleneck. I don't recommend ever completely re-writing your code in C or Cython, but instead find the slow parts and re-implement that in Cython. I always try to optimize my algorithms and use Numpy as much as possible before deciding to use Cython. It's also not the only option - new Just In Time (JIT) compilers are successful for some applications (see [numba](https://numba.pydata.org/)).

## Questions to ask before firing up Cython

* Can I improve the algorithm? $n^2$ -> $n\,\log(n)$?
* Can I optimize my Python code? e.g., use list comprehensions vs. loops...
* Can I replace loops with Numpy array operations?
* Do I really need to speed this up?
* ::sigh:: ok use Cython / Numba

## Setting up the problem

As an example of a place where Cython can really shine, we're going to look at numerical integration of an orbit under the influence of some external gravitational potential. We'll start by implementing everything in Python, and then build up to an optimized Cython version of the code that speeds up the orbit calculation immensely. 

A little more verbosely, we want to numerically solve for $\boldsymbol{x}(t)$ and $\boldsymbol{v}(t)$ given a gravitational potential $\Phi(\boldsymbol{x})$ and initial conditions $\boldsymbol{x}_0$, $\boldsymbol{v}_0$. To do this, we'll use the Leapfrog integration scheme. With the gravitational potential, we'll need to write an expression and function to compute the acceleration $a(\boldsymbol{x})$. With initial conditions and the acceleration, Leapfrog integration works iteratively as follows:

$$
\begin{align}
\boldsymbol{x}_i &= \boldsymbol{x}_{i-1} + \boldsymbol{v}_{i-1/2}\,\Delta t\\
\boldsymbol{a}_i &= \boldsymbol{a}(\boldsymbol{x}_i)\\
\boldsymbol{v}_{i+1/2} &= \boldsymbol{v}_{i-1/2} + \boldsymbol{a}_i\,\Delta t
\end{align}
$$

In practice, we will want to store the positions and velocities at the synced timestep $i$. We'll therefore implement this scheme as:

$$
\begin{align}
\boldsymbol{x}_i &= \boldsymbol{x}_{i-1} + \boldsymbol{v}_{i-1/2}\,\Delta t\\
\boldsymbol{a}_i &= \boldsymbol{a}(\boldsymbol{x}_i)\\
\boldsymbol{v}_{i} &= \boldsymbol{v}_{i-1/2} + \boldsymbol{a}_i\,\frac{\Delta t}{2}\\
\boldsymbol{v}_{i+1/2} &= \boldsymbol{v}_{i} + \boldsymbol{a}_i\,\frac{\Delta t}{2}
\end{align}
$$

The acceleration function is just:

$$
\boldsymbol{a}(\boldsymbol{x}) = -\boldsymbol{\nabla} \Phi(\boldsymbol{x})
$$

For this example, we'll use the [Hernquist potential](http://adsabs.harvard.edu/abs/1990ApJ...356..359H), defined as:

$$
\Phi(r) = -\frac{G\,m}{r + c}
$$

The acceleration is therefore:

$$
\boldsymbol{a}(\boldsymbol{x}) = -\frac{G\,m}{(r + c)^2} \, \frac{\boldsymbol{x}}{r}
$$

We'll allow the parameters $m$ and $c$ to be changeable, but start by using:

$$
\begin{align}
m &= 10^{11}~{\rm M}_\odot\\
c &= 1~{\rm kpc}
\end{align}
$$

In [ ]:
# parameters of the gravitational potential
m = 1E11 # Msun
c = 1. # kpc

# value of G in our unit system
_G = G.decompose(units).value

First, we'll write a Python function to compute the acceleration above:

In [ ]:
def hernquist_acceleration(xyz, G, m, c):
    r = np.sqrt(xyz[0]**2 + xyz[1]**2 + xyz[2]**2)
    dPhi_dr = G * m / (r + c)**2
    return -dPhi_dr * xyz / r

Now we'll implement the Leapfrog integration scheme. We need to specify initial conditions, a timestep, and a number of steps to run for. We'll return the time, position, and velocity arrays for the numerical orbit:

In [ ]:
def leapfrog_hernquist(x0, v0, dt, n_steps, hernquist_args=()):
    
    # ensure that the initial conditions are arrays
    x0 = np.array(x0)
    v0 = np.array(v0)
    
    # Create arrays to store positions and velocities at all times
    x = np.zeros((n_steps+1, 3))
    v = np.zeros((n_steps+1, 3))
    t = np.zeros(n_steps+1)
    
    x[0] = x0
    v[0] = v0
    
    # Increment velocity by 1/2 step 
    v_iminus1_2 = v0 + dt/2. * hernquist_acceleration(x0, *hernquist_args)
    for i in range(1, n_steps+1):
        x[i] = x[i-1] + v_iminus1_2 * dt # full step
        
        a_i = hernquist_acceleration(x[i], *hernquist_args)
        v[i] = v_iminus1_2 + a_i * dt/2. # half step
        v_iminus1_2 = v[i] + a_i * dt/2. # another half step
        
        t[i] = t[i-1] + dt
    
    return t, x, v

In [ ]:
# We'll define these so we can reuse them later:
x0 = [10., 0, 0]
v0 = [0, 0.15, 0]
dt = 1.
n_steps = 100000

t, x, v = leapfrog_hernquist(x0=x0, v0=v0,
                             dt=dt, n_steps=n_steps,  
                             hernquist_args=(_G, m, c))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10,5))

axes[0].plot(x[:,0], x[:,1], marker='.', linestyle='none', alpha=0.1)
axes[0].set_xlim(-12, 12)
axes[0].set_ylim(-12, 12)

axes[0].set_xlabel('$x$')
axes[0].set_ylabel('$y$')

# ---

axes[1].plot(v[:,0], v[:,1], marker='.', linestyle='none', alpha=0.1)
axes[1].set_xlim(-0.35, 0.35)
axes[1].set_ylim(-0.35, 0.35)

axes[1].set_xlabel('$v_x$')
axes[1].set_ylabel('$v_y$')

fig.tight_layout()

In [ ]:
%%time
_ = leapfrog_hernquist(x0=x0, v0=v0,
                       dt=dt, n_steps=n_steps, 
                       hernquist_args=(_G, m, c))

This didn't take *that* long (we integrated for a really long time!), but what if we need to integrate thousands of orbits simultaneously? Or have more complicated potential models to use? Or want to do an N-body simulation? 

* Can we improve the algorithm? No - Leapfrog is one of the simplest integration schemes!
* Can we optimize the Python code? Not really
* Can we replace loops with Numpy array operations? No - this is an iterative scheme
* Do I really need to speed this up? Maybe...but let's say yes

# Cython in IPython

In [ ]:
# Enables the Cython "magic" command %%cython
%load_ext Cython

Let's start by writing exactly the same code as above in Cython, and just verify that it successfully compiles:

In [ ]:
%%cython

import numpy as np

# abbreviated function name for laziness...
def acc_cy1(xyz, G, m, c):
    r = np.sqrt(xyz[0]**2 + xyz[1]**2 + xyz[2]**2)
    dPhi_dr = G * m / (r + c)**2
    return -dPhi_dr * xyz / r

# and another abbreviated function name for laziness...
def leapfrog_cy1(x0, v0, dt, n_steps, hernquist_args=()):
   
    # ensure that the initial conditions are arrays
    x0 = np.array(x0)
    v0 = np.array(v0)
    
    # Create arrays to store positions and velocities at all times
    x = np.zeros((n_steps+1, 3))
    v = np.zeros((n_steps+1, 3))
    t = np.zeros(n_steps+1)
    
    x[0] = x0
    v[0] = v0
    
    # Increment velocity by 1/2 step 
    v_iminus1_2 = v0 + dt/2. * acc_cy1(x0, *hernquist_args)
    for i in range(1, n_steps+1):
        x[i] = x[i-1] + v_iminus1_2 * dt # full step
        
        a_i = acc_cy1(x[i], *hernquist_args)
        v[i] = v_iminus1_2 + a_i * dt/2. # half step
        v_iminus1_2 = v[i] + a_i * dt/2. # another half step
        
        t[i] = t[i-1] + dt
    
    return t, x, v

In [ ]:
%%time
leapfrog_cy1(x0=x0, v0=v0,
             dt=dt, n_steps=n_steps, 
             hernquist_args=(_G, m, c))

We didn't change anything, but got a ~10–20% speed improvement! But we're looking for orders of magnitude speed improvement.

So why is this still slow? Even though we compile it to C using Cython, it's still going through the Python layer. Python is written in C, so the generated C code is actually what Python uses to execute any code! One way we can see this is to use Cython's "annotate" feature to look at what lines hit the Python layer (and thus slow us down). This is the exact same code as above, but with `--annotate` added after the Cython magic:

In [ ]:
%%cython --annotate

import numpy as np

def acc_cy1(xyz, G, m, c):
    r = np.sqrt(xyz[0]**2 + xyz[1]**2 + xyz[2]**2)
    dPhi_dr = G * m / (r + c)**2
    return -dPhi_dr * xyz / r

def leapfrog_cy1(x0, v0, dt, n_steps, hernquist_args=()):
   
    # ensure that the initial conditions are arrays
    x0 = np.array(x0)
    v0 = np.array(v0)
    
    # Create arrays to store positions and velocities at all times
    x = np.zeros((n_steps+1, 3))
    v = np.zeros((n_steps+1, 3))
    t = np.zeros(n_steps+1)
    
    x[0] = x0
    v[0] = v0
    
    # Increment velocity by 1/2 step 
    v_iminus1_2 = v0 + dt/2. * acc_cy1(x0, *hernquist_args)
    for i in range(1, n_steps+1):
        x[i] = x[i-1] + v_iminus1_2 * dt # full step
        
        a_i = acc_cy1(x[i], *hernquist_args)
        v[i] = v_iminus1_2 + a_i * dt/2. # half step
        v_iminus1_2 = v[i] + a_i * dt/2. # another half step
        
        t[i] = t[i-1] + dt
    
    return t, x, v

In the above, everything yellow hints at interaction with the Python layer. We can click on a given line to see the C code it generates.

---

So how do we make this faster? Or, how do we remove the interaction with the Python layer? Cython will do this automatically if we write our code to look more like C. This means adding explicit types, and telling Cython to remove some of the nice features of Python like array index bounds checking. Let's start by re-writing our acceleration function to declare types and remove as much Python overhead as possible:

In [ ]:
%%cython --annotate

import numpy as np # access to Numpy from Python layer
cimport numpy as np # access to Numpy from Cython layer

# this is like #include <math.h>, but defines the functions in Cython
from libc.math cimport sqrt

def acc_cy2(np.ndarray[np.float64_t, ndim=1] xyz, double G, double m, double a):
    cdef:
        double r
        double dPhi_dr
        np.ndarray[np.float64_t, ndim=1] a_xyz = np.zeros(3, np.float64)
        
    r = sqrt(xyz[0]**2 + xyz[1]**2 + xyz[2]**2)
    dPhi_dr = G * m / (r + a)**2
    
    a_xyz[0] = -dPhi_dr * xyz[0] / r
    a_xyz[1] = -dPhi_dr * xyz[1] / r
    a_xyz[2] = -dPhi_dr * xyz[2] / r
    
    return a_xyz

So why are so many of the lines still yellow? Let's take a look at one of them. 

Many of the lines call "`RaiseBufferIndexError`". This is because whenever you access the index of a variable from Python, it checks whether the index is valid given the size of the array. This is a nice feature, but adds quite a bit of overhead in the code if you are doing array access a lot. In this example, we are! The acceleration function gets called during every iteration of the loop. We can turn of bounds checking using a special decorator. While we're at it, we'll turn off a few other checks - see the [cython compiler directives](http://cython.readthedocs.io/en/latest/src/reference/compilation.html#compiler-directives) for a full list of things you can control:

In [ ]:
%%cython --annotate

import numpy as np # access to Numpy from Python layer
cimport numpy as np # access to Numpy from Cython layer

# this is like #include <math.h>, but defines the functions in Cython
from libc.math cimport sqrt

cimport cython

@cython.boundscheck(False) # won't check that index is in bounds of array
@cython.wraparound(False) # array[-1] won't work
@cython.nonecheck(False) # variables are never set to None
@cython.cdivision(True) # don't protect against dividing by zero
def acc_cy3(np.ndarray[np.float64_t, ndim=1] xyz, double G, double m, double a):
    cdef:
        double r
        double dPhi_dr
        np.ndarray[np.float64_t, ndim=1] a_xyz = np.zeros(3, np.float64)
        
    r = sqrt(xyz[0]**2 + xyz[1]**2 + xyz[2]**2)
    dPhi_dr = G * m / (r + a)**2
    
    a_xyz[0] = -dPhi_dr * xyz[0] / r
    a_xyz[1] = -dPhi_dr * xyz[1] / r
    a_xyz[2] = -dPhi_dr * xyz[2] / r
    
    return a_xyz

Nice, ok now only a few lines are yellow:
* the function definition
* creating the acceleration array
* returning the acceleration array

To make this function pure C, we will:
* make this a cdef function (only accessible from Cython)
* pass in a pre-defined array to store the acceleration values

In [ ]:
%%cython --annotate

import numpy as np # access to Numpy from Python layer
cimport numpy as np # access to Numpy from Cython layer

# this is like #include <math.h>, but defines the functions in Cython
from libc.math cimport sqrt

cimport cython

@cython.boundscheck(False) # won't check that index is in bounds of array
@cython.wraparound(False) # array[-1] won't work
@cython.nonecheck(False) # variables are never set to None
@cython.cdivision(True) # don't protect against dividing by zero
cdef void acc_cy4(np.ndarray[np.float64_t, ndim=1] xyz, double G, double m, double a,
                  np.ndarray[np.float64_t, ndim=1] a_xyz):
    cdef:
        double r
        double dPhi_dr
        
    r = sqrt(xyz[0]**2 + xyz[1]**2 + xyz[2]**2)
    dPhi_dr = G * m / (r + a)**2
    
    a_xyz[0] = -dPhi_dr * xyz[0] / r
    a_xyz[1] = -dPhi_dr * xyz[1] / r
    a_xyz[2] = -dPhi_dr * xyz[2] / r

The above function definition still expects an array passed in *by value*. If we truly want to optimize the hell out of this function, we instead want to pass in be reference using a special Numpy/Cython type called a **memoryview**. A memoryview is like a pointer to a point in memory, but allows easy access to the data behind Numpy arrays:

In [ ]:
%%cython --annotate

import numpy as np # access to Numpy from Python layer
cimport numpy as np # access to Numpy from Cython layer

# this is like #include <math.h>, but defines the functions in Cython
from libc.math cimport sqrt

cimport cython

@cython.boundscheck(False) # won't check that index is in bounds of array
@cython.wraparound(False) # array[-1] won't work
@cython.nonecheck(False) # variables are never set to None
@cython.cdivision(True) # don't protect against dividing by zero
cdef void acc_cy5(double[::1] xyz, double G, double m, double a,
                  double[::1] a_xyz):
    cdef:
        double r
        double dPhi_dr
        
    r = sqrt(xyz[0]**2 + xyz[1]**2 + xyz[2]**2)
    dPhi_dr = G * m / (r + a)**2
    
    a_xyz[0] = -dPhi_dr * xyz[0] / r
    a_xyz[1] = -dPhi_dr * xyz[1] / r
    a_xyz[2] = -dPhi_dr * xyz[2] / r

No more yellow! This function is now basically pure C, and will have pure-C speeds when called from Cython. Let's now implement the integration function.

What changes do we need to make to the leapfrog function? Mostly, add types to variables, but keep an eye out for:

* Loops in Python syntax but with cdef'd variables will be converted into C loops. This removes the iteration overhead of Python and makes them C-fast
* Array operations no longer work, so we have to explicitly deal with the 3 components of the position/velocity/acceleration
* After filling the position and velocity memoryviews, when we return, we have to call `np.array()` to turn them back into arrays

In [ ]:
%%cython --annotate
# cython: boundscheck=False, wraparound=False, nonecheck=False, cdivision=True

import numpy as np # access to Numpy from Python layer
cimport numpy as np # access to Numpy from Cython layer
np.import_array()

# this is like #include <math.h>, but defines the functions in Cython
from libc.math cimport sqrt

cimport cython

cdef void acc_cy6(double[::1] xyz, double G, double m, double a,
                  double[::1] a_xyz):
    cdef:
        double r
        double dPhi_dr
        
    r = sqrt(xyz[0]**2 + xyz[1]**2 + xyz[2]**2)
    dPhi_dr = G * m / (r + a)**2
    
    a_xyz[0] = -dPhi_dr * xyz[0] / r
    a_xyz[1] = -dPhi_dr * xyz[1] / r
    a_xyz[2] = -dPhi_dr * xyz[2] / r
    
cpdef leapfrog_cy6(x0, v0, double dt, int n_steps, hernquist_args=()):
    cdef:
        # define memoryview's for initial conditions
        double[::1] _x0 = np.array(x0, np.float64)
        double[::1] _v0 = np.array(v0, np.float64)
    
        # Create arrays to store positions and velocities at all times
        double[:,::1] x = np.zeros((n_steps+1, 3), np.float64) # 2d arrays - note the [:,::1]
        double[:,::1] v = np.zeros((n_steps+1, 3), np.float64)
        double[::1] t = np.zeros(n_steps+1, np.float64)
        
        # Explicitly type the iteration variable 
        int i, k
        
        # placeholder for acceleration values
        double[::1] a_i = np.zeros(3) 
        
        # placeholder for velocity incremented by 1/2 step
        double[::1] v_iminus1_2 = np.zeros(3) 
        
        # explicitly typed and defined parameters
        double G = float(hernquist_args[0])
        double m = float(hernquist_args[1])
        double c = float(hernquist_args[2])        
    
    # get the acceleration at the initial position
    acc_cy6(_x0, G, m, c, a_i)
    
    # if i is cython typed, this will be a much more efficient C loop
    for k in range(3):
        x[0,k] = _x0[k]
        v[0,k] = _v0[k]
    
        # Increment velocity by 1/2 step 
        v_iminus1_2[k] = _v0[k] + dt/2. * a_i[k]
        
    for i in range(1, n_steps+1):
        for k in range(3):
            x[i,k] = x[i-1,k] + v_iminus1_2[k] * dt # full step
        
        acc_cy6(x[i], G, m, c, a_i)
        
        for k in range(3):
            v[i,k] = v_iminus1_2[k] + a_i[k] * dt/2. # half step
            v_iminus1_2[k] = v[i,k] + a_i[k] * dt/2. # another half step
        
        t[i] = t[i-1] + dt
    
    # convert from memoryview to array
    return np.array(t), np.array(x), np.array(v)

In [ ]:
%%time
t_cy, x_cy, v_cy = leapfrog_cy6(x0=x0, v0=v0,
                                dt=dt, n_steps=n_steps, 
                                hernquist_args=(_G, m, c))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10,5))

axes[0].plot(x_cy[:,0], x_cy[:,1], marker='.', linestyle='none', alpha=0.1)
axes[0].set_xlim(-12, 12)
axes[0].set_ylim(-12, 12)

axes[0].set_xlabel('$x$')
axes[0].set_ylabel('$y$')

# ---

axes[1].plot(v_cy[:,0], v_cy[:,1], marker='.', linestyle='none', alpha=0.1)
axes[1].set_xlim(-0.35, 0.35)
axes[1].set_ylim(-0.35, 0.35)

axes[1].set_xlabel('$v_x$')
axes[1].set_ylabel('$v_y$')

fig.tight_layout()

A >200 times speedup over the initial Python-only implementation! 

We're now going to use this same example in a standalone module, and as a part of a package to show other ways to compile Cython.